### Set Up

In [2]:
import pandas as pd
import numpy as np
import datetime

power_raw = pd.read_csv('Lottery_Powerball_Winning_Numbers__Beginning_2010.csv', encoding="Latin-1")
power_raw = power_raw.drop(columns=['Multiplier'])

#format changes:
# 2009-01-04 (drawn on 1-7) changed to 5/59 - 1/39
# 2012-01-15 changed to 1/35 (drawn on 1-18) 
# index 204

# 2015-10-04 changed to 5/69 - 1/26 (drawn on 10-7)
# index 591

#print(power_raw[power_raw['Draw Date']=='10/07/2015'].index.values)

power_first = power_raw.loc[:203, :]
power_second = power_raw.loc[204:590, :]
power_third = power_raw.loc[591:, :]

### Lottery Frequency and Date Functions

In [3]:
# This function will split the winning numbers column into columns each with only one number
def split_cols(df, name1, name2, name3, name4, name5, name6):
    df[[name1, name2, name3, name4, name5, name6]] = df['Winning Numbers'].str.split(' ', expand = True).astype(int) 
    df= df.drop(columns=['Winning Numbers'])
    return df   

# This function will return dataframe on Mega Ball only
def powerball(df):
    df = df.iloc[:,[0,6]].copy()
    df['Draw Date'] = pd.to_datetime(df['Draw Date'])
    df = df.sort_values(by='Draw Date', ascending=True)
    return df

# This function will return dataframe on White Ball only
def whiteball(df, ball_type):
    df = df.drop(columns='Power Ball',)
    df = pd.melt(df, id_vars=['Draw Date'], var_name='Ball Order', value_name=ball_type)
    df = df.drop(columns = ['Ball Order'])
    df['Draw Date'] = pd.to_datetime(df['Draw Date'])
    df = df.sort_values(by='Draw Date', ascending=True)
    return df

# This function will return a frequency count for each number
def ballcount(df, col_name, count_name):
    df = df[col_name].value_counts().rename_axis(col_name).reset_index(name=count_name)
    return df

# This function would return the last drawn date for each number
def checkdate(df):
    emptydict = {}
    for i, j in df.iterrows():
        date = j[0]
        number = j[1]
        emptydict[number] = date
    return emptydict

def datelastdrawn(df1, df2, ball_type, date):
    # Merging the frequency dataframe with date dataframe
    df3 = df1.merge(df2, left_on = ball_type, right_on = ball_type, how= 'right')
    
    # Calculating days since the number has been last drawn
    if date == 'Now':
        now = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        df3['Days Since Last Drawing'] = now - df3['Date']
    else:
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        df3['Days Since Last Drawing'] = date - df3['Date']
    return df3

### Powerball frequency - 01/15/2012

In [4]:
first_df = power_first.copy()

firstdf = split_cols(first_df, 'First', 'Second', 'Third', 'Fourth', 'Fifth', 'Power Ball')
first_white = whiteball(firstdf, 'White Ball')
first_power = powerball(firstdf)

first_whitecount = ballcount(first_white, 'White Ball', 'White Count')
first_powercount = ballcount(first_power, 'Power Ball', 'Power Count')

first_whitedate = pd.DataFrame(checkdate(first_white).items(), columns=['White Ball', 'Date'])
first_powerdate = pd.DataFrame(checkdate(first_power).items(), columns=['Power Ball', 'Date'])

lastdate = firstdf['Draw Date'].iloc[-1]
first_whitecountdate = datelastdrawn(first_whitecount, first_whitedate, 'White Ball', lastdate)
first_powercountdate = datelastdrawn(first_powercount, first_powerdate, 'Power Ball', lastdate)

first_power_sorted = first_powercountdate.sort_values(by = 'Days Since Last Drawing')
first_white_sorted = first_whitecountdate.sort_values(by = 'Days Since Last Drawing')

first_white_sorted.head()

,White Ball,White Count,Date,Days Since Last Drawing
1,41,28,2012-01-14,0 days
22,30,19,2012-01-14,0 days
6,36,23,2012-01-14,0 days
8,38,21,2012-01-14,0 days
9,10,21,2012-01-14,0 days


### Powerball frequency 01/18/2012 - 10/04/2015

In [5]:
second_df = power_second.copy()

seconddf = split_cols(second_df, 'First', 'Second', 'Third', 'Fourth', 'Fifth', 'Power Ball')
second_white = whiteball(seconddf, 'White Ball')
second_power = powerball(seconddf)

second_whitecount = ballcount(second_white, 'White Ball', 'White Count')
second_powercount = ballcount(second_power, 'Power Ball', 'Power Count')

second_whitedate = pd.DataFrame(checkdate(second_white).items(), columns=['White Ball', 'Date'])
second_powerdate = pd.DataFrame(checkdate(second_power).items(), columns=['Power Ball', 'Date'])

lastdate = seconddf['Draw Date'].iloc[-1]
second_whitecountdate = datelastdrawn(second_whitecount, second_whitedate, 'White Ball', lastdate)
second_powercountdate = datelastdrawn(second_powercount, second_powerdate, 'Power Ball', lastdate)

second_power_sorted = second_powercountdate.sort_values(by = 'Days Since Last Drawing')
second_white_sorted = second_whitecountdate.sort_values(by = 'Days Since Last Drawing')

second_white_sorted.head()

,White Ball,White Count,Date,Days Since Last Drawing
31,46,33,2015-10-03,0 days
33,6,32,2015-10-03,0 days
41,33,31,2015-10-03,0 days
17,44,35,2015-10-03,0 days
16,26,36,2015-10-03,0 days


### Powerball frequency 10/07/2015 - 05/02/2020

In [6]:
third_df = power_third.copy()

thirddf = split_cols(third_df, 'First', 'Second', 'Third', 'Fourth', 'Fifth', 'Power Ball')
third_white = whiteball(thirddf, 'White Ball')
third_power = powerball(thirddf)

third_whitecount = ballcount(third_white, 'White Ball', 'White Count')
third_powercount = ballcount(third_power, 'Power Ball', 'Power Count')

third_whitedate = pd.DataFrame(checkdate(third_white).items(), columns=['White Ball', 'Date'])
third_powerdate = pd.DataFrame(checkdate(third_power).items(), columns=['Power Ball', 'Date'])

lastdate = thirddf['Draw Date'].iloc[-1]
third_whitecountdate = datelastdrawn(third_whitecount, third_whitedate, 'White Ball', lastdate)
third_powercountdate = datelastdrawn(third_powercount, third_powerdate, 'Power Ball', lastdate)

third_power_sorted = third_powercountdate.sort_values(by = 'Days Since Last Drawing')
third_white_sorted = third_whitecountdate.sort_values(by = 'Days Since Last Drawing')

third_white_sorted.head()

,White Ball,White Count,Date,Days Since Last Drawing
68,58,24,2020-05-02,0 days
62,13,27,2020-05-02,0 days
8,68,41,2020-05-02,0 days
32,33,35,2020-05-02,0 days
16,16,39,2020-05-02,0 days
